In [0]:
from pyspark.sql.types import DateType
from pyspark.sql.types import TimestampType
from datetime import datetime

# Log Maintain Function

In [0]:
class LogTable:
    schemaName = 'FACT'
    tableName = 'log_table'     
    def load(load_type, status, load_to_table_name : str, comments='', created_by='Gokarna Adhikari'):
        id = str(uuid.uuid4())
        LogTable.load_table(     
            LogTable.schemaName,
            LogTable.schemaName, 
            id = id,
            load_type = load_type,
            table_name = load_to_table_name,
            process_start_time = str(datetime.now()),
            process_end_time = str(datetime.now()),
            status = status,
            comments = comments,
            start_date_time = str(datetime.now()),
            end_date_time = str(datetime.now()),
            created_on = str(datetime.now()),
            created_by = created_by
                      )
        return id
        
        
    def load_table(schema : str, table : str, **kwargs):
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema};")
        json_rdd = spark.sparkContext.parallelize([kwargs])
        df = spark.read.json(json_rdd)

        # converting to compatible type
        timestamp_cols = ["process_start_time", "process_end_time", "start_date_time", "end_date_time", "created_on"]
        for col_name in timestamp_cols:
            df = df.withColumn(col_name, col(col_name).cast(TimestampType()))
            
        if spark.catalog.tableExists(f"{schema}.{LogTable.tableName}"):
            df.write.format("delta").mode("append").saveAsTable(f"{schema}.{LogTable.tableName}")
        else:
            df.write.format("delta").mode("overwrite").option("path", "/path/to/table").saveAsTable(f"{schema}.{LogTable.tableName}")

#         df.write.format('delta').mode('append').option("mergeSchema", "true").saveAsTable(f"{schema}.{LogTable.tableName}")
        
        return kwargs['id']


In [0]:
# %sql
# DROP TABLE FACT.log_table